In [1]:
from pathlib import Path
import sys
sys.path.append('../')
from quantized_fast_forward.fast_forward.index import H5Index, Mode
from quantized_fast_forward.fast_forward.encoder import QueryEncoder
from quantized_fast_forward.fast_forward.ranking import Ranking
from pyserini.search.faiss import AggretrieverQueryEncoder
import numpy as np

/home/catalinlup/anaconda3/envs/ranking/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# define the query encoders
class FFAggretrieverQueryEncoder(QueryEncoder):
    def __init__(self, model_name, device="cpu"):
        self._enc = AggretrieverQueryEncoder(model_name, device=device)

    # for whatever reason, pyserini flattens the outputs, so we need this
    # TODO: implement batch encoding
    def encode(self, queries):
        return np.array([self._enc.encode(q) for q in queries])

In [3]:
# define the index
index = H5Index.from_disk(
    Path("/home/catalinlup/MyWorkspace/MasterThesis/datasets/h5_indices/faiss.msmarco-v1-passage.aggretriever-cocondenser.h5"),
    FFAggretrieverQueryEncoder("castorini/aggretriever-cocondenser"),
)

In [8]:
# load the queries
from datasets import get_dataset
queries, qrel = get_dataset('dev')

queries_psg20 = {x.query_id: x.text for x in queries}


# psg20 = ir_datasets.load("msmarco-passage/trec-dl-2020")
# queries_psg20 = {x.query_id: x.text for x in psg20.queries_iter()}

/home/catalinlup/MyWorkspace/MasterThesis/ExperimentsCodebase/experiment_notebooks/datasets.py:60: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  queries_list.append(GenericQuery(str(queries.iloc[i][0]), queries.iloc[i][1]))
/home/catalinlup/MyWorkspace/MasterThesis/ExperimentsCodebase/experiment_notebooks/datasets.py:63: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  qrels_list.append(TrecQrel(query_id=str(qrels.iloc[i][0]), doc_id=str(qrels.iloc[i][2]), relevance=int(qrels.iloc[i][3]), iteration=str(qrels.iloc[i][1])))


In [9]:
# load the ranking
bm25_ranking = Ranking.from_file(Path("/home/catalinlup/MyWorkspace/MasterThesis/datasets/run_files/msmarco_psg_bm25_rankings.tsv"))
print(f"loaded run with {len(bm25_ranking)} queries")

# print(splade_ranking)

for q_id in bm25_ranking.q_ids:
    assert q_id in queries_psg20

loaded run with 200 queries


AssertionError: 

In [6]:
# run the re-ranking experiment
alpha = 0.3
result = index.get_scores(
    bm25_ranking,
    queries_psg20,
    alpha=alpha,
)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [02:27<00:00,  1.36it/s]


In [7]:
from ir_measures import calc_aggregate, nDCG, AP, RR

# print(result)

METRICS = [nDCG@10, AP(rel=1)@1000, RR(rel=1)@10]

print(
    "BM25",
    calc_aggregate(METRICS, qrel, bm25_ranking.run),
)
print(
    f"BM25, Aggretriever (alpha={alpha})",
    calc_aggregate(METRICS, qrel, result[alpha].run),
)

BM25 {nDCG@10: 0.4875508583120808, RR(rel=2)@10: 0.6554012345679012, AP(rel=2)@1000: 0.2875982310640567}
BM25, Aggretriever (alpha=0.3) {nDCG@10: 0.7165937624269141, RR(rel=2)@10: 0.8252057613168725, AP(rel=2)@1000: 0.4854707093261463}


In [8]:
# run the re-ranking experiment without any interpolation
alpha2 = 0.0
result2 = index.get_scores(
    bm25_ranking,
    queries_psg20,
    alpha=alpha2,
)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [02:23<00:00,  1.40it/s]


In [9]:
print(
    f"BM25, Aggretriever (alpha={alpha2})",
    calc_aggregate(METRICS, psg20.qrels_iter(), result2[alpha2].run),
)

BM25, Aggretriever (alpha=0.0) {nDCG@10: 0.7041666136998518, RR(rel=2)@10: 0.8231481481481481, AP(rel=2)@1000: 0.48485635553462436}
